In [1]:
import os

import pyttsx3
from espeakng import ESpeakNG
from gtts import gTTS

import abox
from config import *

def ttsGTTS(text: str):
    """
     limit are not known but probaly are similar to the official-api 
     see https://cloud.google.com/text-to-speech/quotas
     see also https://github.com/pndurette/gTTS/issues/117#issuecomment-458634045
    """
    tts = gTTS(text, lang='de')
    tts.save(os.path.join(BASE_DIR, 'data_generation', "gTTS.mp3"))


# TODO not sure about the quality of espeak-ng
# requires espeak-ng  to be installed locally
def ttsEsng(text: str):
    esng = ESpeakNG()
    esng.voice = 'german'
    esng.voice = 'German'
    # esng.pitch = 80
    # esng.speed = 120
    # esng.volume = 200
    #for v in esng.voices:
        #print(v)
    wavs = esng.synth_wav(text)
    if wavs:
        print("present")
        with open(os.path.join(BASE_DIR, 'data_generation', "esng.wav"), 'wb') as output:
            output.write(wavs)
    else:
        print("nope")
    # wav = wave.open(StringIO.StringIO(wavs))
    # print(wav.getnchannels(), wav.getframerate(), wav.getnframes())


# TODO https://acapela-box.com/AcaBox/index.php does not serve an api.
#  For a cloud base api see https://www.acapela-group.com/solutions/acapela-vaas/
# TODO not sure about this implementation -> needs python-pympv or other library that provides codecs
# def ttsAbox(text: str):
#     box = abox.Abox(output=os.path.join(BASE_DIR, 'data_generation', "abox.ogg"), voice="willbadguy22k")
#     url = box.query(text)
#     ret = abox.ab_download(url, box.option_l)
#     if not ret:
#         print("Aborting")


def run(text: str):
    # ttsGTTS(text)
    ttsEsng(text)
    # ttsAbox(text)


run("Hallo Welt das hier ist ein deutscher Text gesprochen von einem Programm.")


present


In [1]:
# based on https://cloud.google.com/text-to-speech/docs/reference/libraries#cloud-console
# for pricing see https://cloud.google.com/text-to-speech/pricing
# for languages/voices see https://cloud.google.com/text-to-speech/docs/voices

import os
from google.oauth2 import service_account
from config import *
from google.cloud import texttospeech

os.makedirs(name=os.path.join(BASE_DIR, "data_generation", "google_cloud"),exist_ok=True)

# Instantiates a client
credentials = service_account.Credentials.from_service_account_file(GOOGLE_APPLICATION_CREDENTIALS)
client = texttospeech.TextToSpeechClient(credentials=credentials)

# Set the text input to be synthesized
text="Hallo Welt das hier ist ein deutscher Text gesprochen von einem Programm."
synthesis_input = texttospeech.types.SynthesisInput(text=text)

# Build the voice request, select the language and voice
# Standard Female
voice = texttospeech.types.VoiceSelectionParams(language_code="de-DE",name="de-DE-Standard-A")
# Standard Male
voice = texttospeech.types.VoiceSelectionParams(language_code="de-DE",name="de-DE-Standard-B")
# Neutral Ssml => seems to default to de-DE-Standard-A
voice = texttospeech.types.VoiceSelectionParams(language_code="de-DE",ssml_gender=texttospeech.enums.SsmlVoiceGender.NEUTRAL)
# Select the type of audio file you want returned
audio_config = texttospeech.types.AudioConfig(audio_encoding=texttospeech.enums.AudioEncoding.LINEAR16)
# Perform the text-to-speech request on the text input with the selected voice parameters and audio file type
response = client.synthesize_speech(synthesis_input, voice, audio_config)

# The response's audio_content is binary.
with open(os.path.join(BASE_DIR, "data_generation", "google_cloud","google_cloud_n.wav"), 'wb') as out:
    # Write the response to the output file.
    out.write(response.audio_content)
    print('Audio content written to file')

Audio content written to file
Audio content written to file
Audio content written to file


In [7]:
from pydub import AudioSegment
import io
ac=response.audio_content
podcast = AudioSegment.from_wav( io.BytesIO(ac))
podcast.export("testme.flac",format = "flac")

<_io.BufferedRandom name='testme.flac'>

In [43]:
# based on https://azure.microsoft.com/en-us/pricing/details/cognitive-services/speech-services/
# for pricing see https://azure.microsoft.com/en-us/pricing/details/cognitive-services/speech-services/
# for voices see https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/language-support#standard-voices

# NOTE: on manjaro this sdk does not seem to work as some libary are not present?
# -> this may also happen on newer linux distribution in case the packages/kernel are updated
import os
import azure.cognitiveservices.speech as speechsdk
from config import *

os.makedirs(name=os.path.join(BASE_DIR, "data_generation", "azure_cloud"),exist_ok=True)

# Replace with your own subscription key and region identifier from here: https://aka.ms/speech/sdkregion
speech_config = speechsdk.SpeechConfig(subscription=AZURE_APPLICATION_KEY, region=AZURE_APPLICATION_REGION)

# Creates an audio configuration that points to an audio file.
# Replace with your own audio filename.
audio_filename = os.path.join(BASE_DIR, "data_generation", "azure_cloud","helloworld.mp3")
audio_output = speechsdk.audio.AudioOutputConfig(filename=audio_filename)

# Creates a synthesizer with the given settings
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_output)

# Synthesizes the text to speech.
# Replace with your own text.
text = "Hello world!"
result = speech_synthesizer.speak_text_async(text).get()

# Checks result.
if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
    print("Speech synthesized to [{}] for text [{}]".format(audio_filename, text))
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("Speech synthesis canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        if cancellation_details.error_details:
            print("Error details: {}".format(cancellation_details.error_details))
            print(cancellation_details)
    print("Did you update the subscription info?")

Speech synthesis canceled: CancellationReason.Error
Error details: Connection failed (no connection to the remote host). Internal error: 1. Error details: 2460. Please check network connection, firewall setting, and the region name used to create speech factory. USP state: 2. Received audio size: 0bytes.
SpeechSynthesisCancellationDetails(reason=CancellationReason.Error, error_details="Connection failed (no connection to the remote host). Internal error: 1. Error details: 2460. Please check network connection, firewall setting, and the region name used to create speech factory. USP state: 2. Received audio size: 0bytes.")
Did you update the subscription info?


In [40]:
# https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/quickstart-python-text-to-speech

import os
import requests
import time
from xml.etree import ElementTree
from config import *

def get_token():
    fetch_token_url = 'https://'+AZURE_APPLICATION_REGION+'.api.cognitive.microsoft.com/sts/v1.0/issueToken'
    headers = {
        'Ocp-Apim-Subscription-Key': AZURE_APPLICATION_KEY
    }
    response = requests.post(fetch_token_url, headers=headers)
    access_token = str(response.text)
    return access_token

basePath=os.path.join(BASE_DIR, "data_generation", "azure_cloud")
os.makedirs(name=basePath,exist_ok=True)

base_url = 'https://'+AZURE_APPLICATION_REGION+'.tts.speech.microsoft.com/'
path = 'cognitiveservices/v1'
constructed_url = base_url + path
headers = {
    'Authorization': 'Bearer ' + get_token(),
    'Content-Type': 'application/ssml+xml',
    # see https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/rest-text-to-speech#audio-outputs
    'X-Microsoft-OutputFormat': 'riff-16khz-16bit-mono-pcm',
    'User-Agent': 'YOUR_RESOURCE_NAME'
}

In [39]:
voiceName="de-DE-Hedda"

xml_body = ElementTree.Element('speak', version='1.0')
xml_body.set('{http://www.w3.org/XML/1998/namespace}lang', 'en-US')
voice = ElementTree.SubElement(xml_body, 'voice')
voice.set('{http://www.w3.org/XML/1998/namespace}lang', 'en-US')
voice.set('name', voiceName)
voice.text = "Hallo Welt das hier ist ein deutscher Text gesprochen von einem Programm."
body = ElementTree.tostring(xml_body)
# TODO refactor + test this code here.
response = requests.post(constructed_url, headers=headers, data=body)
if response.status_code == 200:
    with open(os.path.join(basePath,"sample-"+voiceName+".mp3"), 'wb') as audio:
        audio.write(response.content)
        print("\nStatus code: " + str(response.status_code) +
              "\nYour TTS is ready for playback.\n")
else:
    print("\nStatus code: " + str(response.status_code) +
          "\nSomething went wrong. Check your subscription key and headers.\n")


Status code: 200
Your TTS is ready for playback.



In [31]:
"""Getting Started Example for Python 2.7+/3.3+"""
from boto3 import Session
from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import subprocess
from tempfile import gettempdir
from pydub import AudioSegment
import io
from config import *

basePath=os.path.join(BASE_DIR, "data_generation", "aws")

# Create a client using the credentials and region defined in the [adminuser]
# section of the AWS credentials file (~/.aws/credentials).
session = Session(profile_name="adminuser")
polly = session.client("polly")

try:
    # Request speech synthesis
    response = polly.synthesize_speech(Text="Hello world!", OutputFormat="pcm", SampleRate='16000',VoiceId="Joanna")
except (BotoCoreError, ClientError) as error:
    # The service returned an error, exit gracefully
    print(error)
    sys.exit(-1)

# Access the audio stream from the response
if "AudioStream" in response:
    # Note: Closing the stream is important because the service throttles on the
    # number of parallel connections. Here we are using contextlib.closing to
    # ensure the close method of the stream object will be called automatically
    # at the end of the with statement's scope.m:
    with closing(response["AudioStream"]) as stream:
        result=stream.read()
        try:
            # Open a file for writing the output as a binary stream
            with open( os.path.join(basePath,"speech2.raw"), "wb") as file:
                file.write(result)
            # convert aws pcm see https://jun711.github.io/aws/convert-aws-polly-synthesized-speech-from-pcm-to-wav-format/
            podcast=AudioSegment(result, sample_width=2, frame_rate=16000, channels=1)
            podcast.export( os.path.join(basePath,"testme2.flac"),format = "flac")
        except IOError as error:
            # Could not write to file, exit gracefully
            print(error)
            sys.exit(-1)

else:
    # The response didn't contain audio data, exit gracefully
    print("Could not stream audio")
    sys.exit(-1)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



[Errno 2] No such file or directory: '/home/cintoros/Documents/git_repos/speech-collection-app/data/data_generation/aws/speech2.raw'
Traceback (most recent call last):
  File "<ipython-input-31-11a32be1beba>", line 38, in <module>
    with open( os.path.join(basePath,"speech2.raw"), "wb") as file:
FileNotFoundError: [Errno 2] No such file or directory: '/home/cintoros/Documents/git_repos/speech-collection-app/data/data_generation/aws/speech2.raw'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/cintoros/miniconda3/envs/speech_collection_app_data_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-31-11a32be1beba>", line 46, in <module>
    sys.exit(-1)
SystemExit: -1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/cintoros/mini

SystemExit: -1

/home/cintoros/miniconda3/envs/speech_collection_app_data_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


<_io.BufferedRandom name='testme.flac'>

In [ ]:
from data_generation import SequentualApiFetcher
api_fetcher= SequentualApiFetcher()
api_fetcher.request_all()

In [1]:
import os;
from mutagen.mp3 import MP3
from config import *
seconds=0
count=0
errors=[]
entries = os.scandir()
for fileData in os.listdir(os.path.join(BASE_DIR, "data_generation", "sequentual")):
    try:
        count+=1;
        audio = MP3(os.path.join(os.path.join(BASE_DIR, "data_generation", "sequentual",fileData)))
        seconds+=audio.info.length
    except:
        errors.append(fileData)

print("sec: ",seconds ,"h: ",seconds/60/60,"count: ",count,"errors: ",len(errors))
# TODO delete files with errors from filesystem and database

sec:  792958.6535000033 h:  220.2662926388898 count:  188168 errors:  126


In [18]:
import mysql.connector
connection = mysql.connector.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
cursor = connection.cursor(dictionary=True)
connection.autocommit = True
for error in errors:
    (id,voice)=error.split("_",1)
    voice=voice.replace(".mp3","")
    cursor.execute("DELETE FROM generated_audio WHERE text_id=%s and voice=%s",[id,voice])
    pe=os.path.join(BASE_DIR, "data_generation", "sequentual",error)
    os.remove(pe);